In [203]:
%pip install numpy
%pip install pandas
%pip install plotly
%pip install nbformat


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [204]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import nbformat

In [205]:
accidents = pd.read_csv("Road Accident Data.csv")

/var/folders/ql/ql4mz26x195c_g5_8yx6372h0000gn/T/ipykernel_63322/3785209320.py:1: DtypeWarning:

Columns (6,10,12,16) have mixed types. Specify dtype option on import or set low_memory=False.



In [206]:
#Replace 'Fetal' with 'Fatal' in the Accident_Severity column
accidents['Accident_Severity'] = accidents['Accident_Severity'].replace('Fetal', 'Fatal')


In [207]:
# Convert the Accident_Index column to sequential numbers for convenience
accidents["Accident_Index"] = range(1, len(accidents) + 1)

In [208]:
def ensure_date_format(column, desired_format='%Y-%m-%d', input_format='%m/%d/%Y'):
    formatted_dates = []
    for date in column:
        try:
            # Попробуем привести дату к желаемому формату
            formatted_date = pd.to_datetime(date, format=desired_format).strftime(desired_format)
        except ValueError:
            # Если формат не совпадает, преобразуем из исходного формата
            formatted_date = pd.to_datetime(date, format=input_format).strftime(desired_format)
        formatted_dates.append(formatted_date)
    return formatted_dates

# Применяем функцию к столбцу с датами
accidents['Accident Date'] = ensure_date_format(accidents['Accident Date'])

In [209]:
severity_mapping = {'Slight': 1, 'Serious': 2, 'Fatal': 3}
accidents['Severity_Numeric'] = accidents['Accident_Severity'].map(severity_mapping)

In [210]:
#transformation into day and night
daylight_conditions = ['Daylight', 'Daylight: street light present']
accidents['Day_or_Night'] = accidents['Light_Conditions'].apply(
    lambda x: 'Day' if x in daylight_conditions else 'Night'
)

In [211]:


# We calculate the number of accidents of each severity level
severity_counts = accidents['Accident_Severity'].value_counts()

# create a pie chart
fig = go.Figure(data=[
    go.Pie(
        labels=severity_counts.index,   # Severity categories (Slight, Serious, Fatal)
        values=severity_counts.values, # Number of accidents for each category
        hole=0.0,                      
        textinfo='label+percent',      
        marker=dict(colors=['rgb(135, 206, 250)', 'rgb(255, 165, 0)', 'rgb(255, 69, 0)']) 
    )
])


fig.update_layout(
    title='Emergency severity distribution',
    template='plotly_white'
)


fig.show()


In [212]:
time_of_day_counts = accidents['Day_or_Night'].value_counts()


fig = go.Figure(data=[
    go.Bar(
        x=time_of_day_counts.index,  # Times of Day (Day, Night)
        y=time_of_day_counts.values, # Number of accidents for each time of day
        marker=dict(color=['skyblue', 'orange'])
    )
])


fig.update_layout(
    title='Number of accidents at different times of the day',
    xaxis_title='Times of Day',
    yaxis_title='Number of accidents',
    template='plotly_white'
)


fig.show()

In [213]:
# We count the number of accidents for each type of vehicle
vehicle_type_counts = accidents['Vehicle_Type'].value_counts()

fig = go.Figure(data=[
    go.Bar(
        x=vehicle_type_counts.index,  # Types of vehicles
        y=vehicle_type_counts.values, # Number of accidents for each type
        marker=dict(color='lightblue')
    )
])

fig.update_layout(
    title='Number of accidents by vehicle type',
    xaxis_title='Vehicle type',
    yaxis_title='Number of Accident',
    template='plotly_white',
    xaxis_tickangle=-45  
)

fig.show()


<b>My hypothesis is: - The number of accidents is higher in bad weather conditions and poor lighting<b>


Let's check the average number of casualties under different weather conditions

In [214]:
# Average number of casualties by weather conditions
casualties_weather = accidents.groupby('Weather_Conditions')['Number_of_Casualties'].mean().sort_values()
fig1 = go.Figure()
fig1.add_trace(go.Scatter(
    x=casualties_weather.index,
    y=casualties_weather,
    mode='lines+markers',
    line=dict(color='blue', width=2),
    marker=dict(size=8)
))

fig1.update_layout(
    title="Average number of casualties by weather conditions",
    xaxis_title="Weather conditions",
    yaxis_title="Average number of victims",
    xaxis=dict(tickangle=45),
    template='plotly'
)
fig1.show()

Now let’s check the same ratio, but as a percentage

In [215]:


# Classification of weather conditions into “good” and “bad”
good_weather = ["Fine no high winds", "Fine with high winds"]
bad_weather = ["Rain", "Snow", "Fog or mist", "Other", "Strong winds"]

# Adding the category "Weather_Type"
accidents['Weather_Type'] = accidents['Weather_Conditions'].apply(
    lambda x: 'Good' if x in good_weather else 'Bad' if x in bad_weather else 'Unknown'
)

# Filter data for known weather conditions
filtered_data = accidents[accidents['Weather_Type'] != 'Unknown']

# Grouping to analyze the proportion of serious accidents
severity_counts = filtered_data.groupby(['Weather_Type', 'Accident_Severity']).size().unstack(fill_value=0)
severity_proportions = severity_counts.div(severity_counts.sum(axis=1), axis=0)


# Accident severity distribution
fig2 = go.Figure()
for severity in severity_proportions.columns:
    fig2.add_trace(go.Bar(
        x=severity_proportions.index,
        y=severity_proportions[severity],
        name=severity
    ))

fig2.update_layout(
    title="Distribution of accident severity by weather conditions",
    xaxis_title="Type of weather conditions",
    yaxis_title="Percentage of total accidents",
    barmode='stack',
    legend_title="Severity of the accident",
    template='plotly'
)


fig2.show()


In [216]:




# Filtering data for analysis
filtered_data = accidents[["Accident_Severity", "Weather_Conditions", "Light_Conditions"]].dropna()

# Highlighting only serious accidents
filtered_data["Is_Serious"] = filtered_data["Accident_Severity"].apply(lambda x: 1 if x == "Serious" else 0)

# Grouping data by weather and light conditions
grouped_data = (
    filtered_data.groupby(["Weather_Conditions", "Light_Conditions"])["Is_Serious"]
    .mean()
    .reset_index()
    .rename(columns={"Is_Serious": "Serious_Accident_Rate"})
)

fig6 = px.bar(
    grouped_data,
    x="Weather_Conditions",
    y="Serious_Accident_Rate",
    color="Light_Conditions",
    barmode="group",
    color_discrete_sequence=px.colors.sequential.Viridis,
    title="Proportion of serious accidents under different weather and light conditions",
    labels={
        "Weather_Conditions": "Weather conditions",
        "Serious_Accident_Rate": "Proportion of serious accidents",
        "Light_Conditions": "Light conditions",
    },
)

# Настройка осей и отображения
fig6.update_layout(
    xaxis=dict(title="Weather conditions", tickangle=45),
    yaxis=dict(title="Proportion of serious accidents"),
    legend_title="Light conditions",
    template="plotly_white",
)
fig6.show()


In [217]:
filtered_data["Is_Fatal"] = filtered_data["Accident_Severity"].apply(lambda x: 1 if x == "Fatal" else 0)

# Grouping data by weather and light conditions
grouped_data = (
    filtered_data.groupby(["Weather_Conditions", "Light_Conditions"])["Is_Fatal"]
    .mean()
    .reset_index()
    .rename(columns={"Is_Fatal": "Serious_Accident_Rate"})
)

fig6 = px.bar(
    grouped_data,
    x="Weather_Conditions",
    y="Serious_Accident_Rate",
    color="Light_Conditions",
    barmode="group",
    color_discrete_sequence=px.colors.sequential.Viridis,
    title="Proportion of serious accidents under different weather and light conditions",
    labels={
        "Weather_Conditions": "Weather conditions",
        "Serious_Accident_Rate": "Proportion of serious accidents",
        "Light_Conditions": "Light conditions",
    },
)

# Настройка осей и отображения
fig6.update_layout(
    xaxis=dict(title="Weather conditions", tickangle=45),
    yaxis=dict(title="Proportion of serious accidents"),
    legend_title="Light conditions",
    template="plotly_white",
)
fig6.show()


In [218]:
filtered_data["Is_Slight"] = filtered_data["Accident_Severity"].apply(lambda x: 1 if x == "Slight" else 0)

# Grouping data by weather and light conditions
grouped_data = (
    filtered_data.groupby(["Weather_Conditions", "Light_Conditions"])["Is_Slight"]
    .mean()
    .reset_index()
    .rename(columns={"Is_Slight": "Serious_Accident_Rate"})
)

fig6 = px.bar(
    grouped_data,
    x="Weather_Conditions",
    y="Serious_Accident_Rate",
    color="Light_Conditions",
    barmode="group",
    color_discrete_sequence=px.colors.sequential.Viridis,
    title="Proportion of serious accidents under different weather and light conditions",
    labels={
        "Weather_Conditions": "Weather conditions",
        "Serious_Accident_Rate": "Proportion of serious accidents",
        "Light_Conditions": "Light conditions",
    },
)

# Настройка осей и отображения
fig6.update_layout(
    xaxis=dict(title="Weather conditions", tickangle=45),
    yaxis=dict(title="Proportion of serious accidents"),
    legend_title="Light conditions",
    template="plotly_white",
)
fig6.show()


<b>Conclusion:<b>

After making calculations, we found out that the number of accidents is higher in good weather, but at the same time, the percentage of minor accidents is higher in bad weather. Also, after plotting the graph, we can notice that more accidents happen at night


In [219]:
print("Accident severity distribution:")
print(severity_counts)
print("\nPercentage distribution of accident severity:")
print(severity_proportions)

Accident severity distribution:
Accident_Severity  Fatal  Serious  Slight
Weather_Type                             
Bad                  114     1131    9248
Good                3268    33654  207574

Percentage distribution of accident severity:
Accident_Severity     Fatal   Serious    Slight
Weather_Type                                   
Bad                0.010864  0.107786  0.881349
Good               0.013366  0.137646  0.848987


In [220]:
accidents.head(-1)


,Accident_Index,Accident Date,Day_of_Week,Junction_Control,Junction_Detail,Accident_Severity,Latitude,Light_Conditions,Local_Authority_(District),Carriageway_Hazards,...,Road_Surface_Conditions,Road_Type,Speed_limit,Time,Urban_or_Rural_Area,Weather_Conditions,Vehicle_Type,Severity_Numeric,Day_or_Night,Weather_Type
0,1,2021-01-01,Thursday,Give way or uncontrolled,T or staggered junction,Serious,51.512273,Daylight,Kensington and Chelsea,NaN,...,Dry,One way street,30,15:11,Urban,Fine no high winds,Car,2,Day,Good
1,2,2021-01-05,Monday,Give way or uncontrolled,Crossroads,Serious,51.514399,Daylight,Kensington and Chelsea,NaN,...,Wet or damp,Single carriageway,30,10:59,Urban,Fine no high winds,Taxi/Private hire car,2,Day,Good
2,3,2021-01-04,Sunday,Give way or uncontrolled,T or staggered junction,Slight,51.486668,Daylight,Kensington and Chelsea,NaN,...,Dry,Single carriageway,30,14:19,Urban,Fine no high winds,Taxi/Private hire car,1,Day,Good
3,4,2021-01-05,Monday,Auto traffic signal,T or staggered junction,Serious,51.507804,Daylight,Kensington and Chelsea,NaN,...,Frost or ice,Single carriageway,30,8:10,Urban,Other,Motorcycle over 500cc,2,Day,Bad
4,5,2021-01-06,Tuesday,Auto traffic signal,Crossroads,Serious,51.482076,Darkness - lights lit,Kensington and Chelsea,NaN,...,Dry,Single carriageway,30,17:25,Urban,Fine no high winds,Car,2,Night,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307969,307970,2022-02-21,Sunday,Data missing or out of range,Not at junction or within 20 metres,Slight,57.232273,Darkness - no lighting,Highland,NaN,...,Frost or ice,Single carriageway,60,3:00,Rural,Fine no high winds,Car,1,Night,Good
307970,307971,2022-02-23,Tuesday,Give way or uncontrolled,T or staggered junction,Slight,57.585044,Daylight,Highland,NaN,...,Frost or ice,Single carriageway,30,9:38,Rural,Fine no high winds,Car,1,Day,Good
307971,307972,2022-02-23,Tuesday,Give way or uncontrolled,T or staggered junction,Serious,57.214898,Darkness - no lighting,Highland,NaN,...,Wet or damp,Single carriageway,60,18:25,Rural,Fine no high winds,Motorcycle over 500cc,2,Night,Good
307972,307973,2022-02-28,Sunday,Give way or uncontrolled,T or staggered junction,Serious,57.57521,Daylight,Highland,Other object on road,...,Wet or damp,Dual carriageway,60,15:45,Rural,Snowing no high winds,Car,2,Day,Unknown
